I tried to just load the mhd files directly before doing any preprocessing, but then the kernel died after 21 images loaded, so I might have to do some preprocessing directly after loading the image and then saving that smaller array. When printing the shape of the ct scans I could also see that each slize was 512*512 but that a different number of slices was used for each ct scan, from 100 to 500, so it could differ quite a lot. I don´t know if the slice number has a certain significance. Looking at this article: https://soundvet.com/portable-ct/understanding-ct-scanner-specifications-slice-count-scan-speed-and-cost-efficiency/ it seems like a whole ct scanner might be set to a certain number of slices, but maybe there are scanners where you can adjust the number of slices? According to this article: https://lbnmedical.com/ct-scan-slice-types/ a CT scanner takes 4 - 640 slices. According to the article it seems like our ct scans with over 100 slices are very detailed, have very high image quality and have been taken with expensive machines - alright. I´m still a bit confused about the irregular number of slices, I would expect powers of 2, maybe 128,256 etc but I´m seeing 204, 157, 139 etc - maybe I´ll figure that one out later. 

Since we only need 20 images anyways for the training, maybe we can load just 20 imaged and train the model in this notebook. Then we can load the model from another notebook and do the validation there. 

In [1]:
#SimpleITK helps us load the mhd files into numpy arrays

import SimpleITK as sitk
import numpy as np
import glob
import os

local_path = '/home/ec2-user/SageMaker/images/'

filenames_train = []
for idx,filename in enumerate(glob.glob(os.path.join(local_path, '*.mhd'))):
    filenames_train.append(filename)
    if(idx == 19):
        break

print(len(filenames_train))

20


In [11]:
import numpy as np
def load_mhd_file(file_path):
    # Read the MHD file using SimpleITK
    image = sitk.ReadImage(file_path)
    
    # Convert it to a NumPy array (H x W x D)
    image_array = sitk.GetArrayFromImage(image)
    
    return image_array

image_arrays = []
for filename in filenames_train:
    image_data = load_mhd_file(filename)
    image_arrays.append(image_data)

I found this tutorial for finetuning a U-net model on SageMaker Neo: https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_neo_compilation_jobs/tensorflow_unet/sagemaker-neo-tf-unet.html

I might need to do some changes to change it to using a 3D U-net model though. Let´s do some googling on 3D U-net or U-net in general and let´s try to learn a little bit about the architecture. 

Let´s start by reading the original U-net paper: https://arxiv.org/abs/1505.04597
"U-Net: Convolutional Networks for Biomedical Image Segmentation"

My notes:
* from the abstract I understand that they created a network architecture that performs very well on a small set of images and is also very fast
* so they talk about cnns have existed for a long time but weren´t that successful because we didn´t have enough data and also the resources to train very large networks. Then there was the imagenet paper where the network had 8 layers and millions of parameters, trained on the imagenet dataset with 1 million training images - very successful supervised learning tasks - and then the networks only got larger from there. 
* typically a cnn has en input of pixels and a desired output label, but in biomedical image tasks we´re more interested in localisation - so we want each pixel to have a label. Having thousands of training images usually is just not possible in biomedical tasks as well. 
* they mention a network that uses a patch-approach. One patch area around each pixel as input with an accompanying label for the pixel. They mention how costly/slow this is, and also the overlapping pixels. They also mention a trade-off between localisation accuracy and the size of the pathches as larger patches require more max-pooling layers that reduce the localisation accuracy. I assume this is because max-pooling layers work by downsampling the resolution of the image.
* so the unet is based on a fully convolutional network architecture with some modifications to be able to process less images but with better precision. Here the pooling operators are replaced upsampling operators. The layers thus increase the resolution of their output. 
* so the unet has an encoder part (the first half of the u - also called the contracting path) and then a decoder part (the second half of the u - also called the expansive path). The encoder does the downsampling and the decoder does the upsampling based on the output of the encoder. The network first progressively downsamples the image and only captures high level features through convolutional layers and max-pooling operations. Each layer has more feature channels/filters to capture increasingly complex patterns. Then there´s the decoder part which upsamples the feature maps to restore the original resolution of the input image. to ensure that relevant information is available during this upsampling steps there are skip connections that go directly from layers in the encoder to the decoder. So the local details comes from the skip connections and the global context from the output of the encoder. 
* They don´t process the whole image at once, but uses over-lapping patches such as in he study they mentioned before. 
* They also mirror/flip the image patches to artifically create more data. Another data augmentation technique they use is elastic deformations which deforms the images in a natural way, kinda as if they were made of rubber/elactic. I think this makes the model better at generalising and less prone to overfitting. Deformations are also common in biomedical images apparently - and creating them artifically like this is quite easy. 
* One common problem in image segmentation tasks are neighboring/touching cells having the same label and the model incorrectly assuming that they are the same object. The authors thus implement a modified loss function where the pixels on the boundary of these touching cells are given a higher weight so that the loss function focuses more on getting those right and thus learns to separate touching cells better. 

I kinda skipped over the rest but I think I get the gist of it. 